# Instructions

Ce notebook ne doit contenir que votre script servant à l'entrainement de votre modèle. Nous devons pouvoir l'exécuter en cliquant sur *Exécution -> Tout exécuter*.

Veuillez également ajouter des commentaires dans votre code pour expliquer ce que vous faites. N'hésitez pas à ajouter des blocs de textes (cliquez sur le bouton *+ Texte* en dessous du menu) pour ajouter plus d'explications.

Vous devrez déposer sur Moodle une archive au format .zip contenant un dossier avec vos noms.

Dans ce dossier, nous devons retrouver les deux notebooks (training et testing) ainsi qu'un nouveau dossier *models* contenant les poids de vos modèles entrainés, et si nécessaire un dossier *datasets* contenant d'autres données utilisée pour effectuer l'apprentissage de vos modèles (données obtenues par récupération sur le web "web scraping"  ou bien augmentation de données "data augmentation"). Si vous effectuez de l'augmentation de données, fournissez aussi le code pour la réaliser dans le notebook.

# 1.1. Importation 

In [1]:
import numpy as np
import pandas as pd
import os
import torch
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer, DataCollatorWithPadding, pipeline, logging
import evaluate

import spacy
from gensim.models import KeyedVectors

np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', precision=3, suppress=False, threshold=1000, formatter=None)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
logging.set_verbosity_error()

c:\Users\odanl\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
1
NVIDIA GeForce RTX 3060 Ti


# 2.1. Augmentation de données par vecteur le plus proche

In [42]:
spacy_model_fr = spacy.load("fr_core_news_sm")
model = KeyedVectors.load_word2vec_format("frWac_non_lem_no_postag_no_phrase_500_skip_cut100.bin", binary=True, unicode_errors="ignore")

In [43]:
def find_nearest_vector(word):
    try:
        similar_words = model.most_similar(word)
        return [w[0] for w in similar_words]
    except KeyError:
        return [word]

# Function to augment dataset
def augmented_vector_data(dataset):
    augmented_sentences = []
    for d in dataset:
        augmented_sentences.append(d['data'])

    for d in dataset:
        sentence = d['data']
        spacy_analysed = spacy_model_fr(sentence)
        augmented_sentence = []

        for token in spacy_analysed:
            if token.pos_ == "NOUN":
                nearest = find_nearest_vector(f"{token.text}")
                augmented_sentence.append(nearest[0])
            else:
                augmented_sentence.append(token.text)

        augmented_sentences.append(' '.join(augmented_sentence))

    ids = list(range(0, len(augmented_sentences)))

    # Prepare DataFrame to save
    data = {
        "": ids,
        "data": augmented_sentences,
        "label": [d['label'] for d in dataset] * 2,
        "target_name": [d['target_name'] for d in dataset] * 2
    }

    # Create a DataFrame
    df = pd.DataFrame(data)
    df.to_csv("augmented_vector_train.csv", index=False)
    print("CSV file created: augmented_vector_train.csv")


# 2.2. Augmentation de données avec model de language

In [44]:
def augmented_mask_data(dataset):
    device = 0 if torch.cuda.is_available() else -1
    lm_unmasker = pipeline('fill-mask', model='allenai/longformer-base-4096', device=device)
    augmented_sentences = []
    for d in dataset:
        augmented_sentences.append(d['data'])

    for j,d in enumerate(dataset):
        sentence = d['data']
        spacy_analysed = spacy_model_fr(sentence)
        augmented_sentence = []

        for i, token in enumerate(spacy_analysed):
            if token.pos_ == "NOUN":
                temp = ' '.join([t.text for t in spacy_analysed[:i]]) + " <mask> " + ' '.join([t.text for t in spacy_analysed[i+1:]])
                output = lm_unmasker(temp)
                augmented_sentence.append(output[0]['token_str'])
            else:
                augmented_sentence.append(token.text)


        print(f"{j+1}/{len(dataset)+1}",' '.join(augmented_sentence))
        augmented_sentences.append(' '.join(augmented_sentence))

    
    ids = list(range(0, len(augmented_sentences)))

    # Prepare DataFrame to save
    data = {
        "": ids,
        "data": augmented_sentences,
        "label": [d['label'] for d in dataset] * 2,
        "target_name": [d['target_name'] for d in dataset] * 2
    }

    # Create a DataFrame
    df = pd.DataFrame(data)
    df.to_csv("augmented_mask_train.csv", index=False)
    print("CSV file created: augmented_mask_train.csv")


# 2.3. Load Dataset

In [ ]:
data_augment_methods = "vector_data" #@param ["basic_data","vector_data","mask_data"]
basic_data_path = 'fake_train.csv'
basic_dataset = Dataset.from_pandas(pd.read_csv(basic_data_path))

match(data_augment_methods):
    case 'basic_data':
        dataset = basic_dataset
    case 'vector_data':
        augmented_file_vector_path = 'augmented_vector_train.csv'
        if not os.path.exists(augmented_file_vector_path):
            augmented_vector_data(basic_dataset)
        dataset = Dataset.from_pandas(pd.read_csv(augmented_file_vector_path))
    case 'mask_data': # à éviter prend bcp de temps :sob: (genre +5h avec ma rtx3060ti)
        augmented_file_mask_path = 'augmented_mask_train.csv'
        if not os.path.exists(augmented_file_mask_path):
            augmented_mask_data(basic_dataset)
        dataset = Dataset.from_pandas(pd.read_csv(augmented_file_mask_path))

ds_train, ds_test = dataset.train_test_split(test_size=0.2).values()

# 3. Transformer model Roberta_small

# 3.1. Définition du model

In [ ]:
id2label = {0: "News", 1: "Fake News"}
label2id = {"News": 0, "Fake News": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    "klue/roberta-small",
    num_labels=2,
    id2label=id2label,
    label2id=label2id
).cuda()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")

def preprocess_function(examples):
    return tokenizer(examples['data'], truncation=True, padding=True, max_length=512)

tokenized_train = ds_train.map(preprocess_function, batched=True)
tokenized_test = ds_test.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# 3.2. Lancement du train pour Roberta

In [ ]:
training_args = TrainingArguments(
    output_dir="defi_3_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    push_to_hub=False,
)

# pour save un backup mais bug sur mon pc
#    save_strategy="epoch",
#    load_best_model_at_end=True,

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# 3.3. Save Model

In [ ]:
trainer.save_model("saved_model_rob_small")
tokenizer.save_pretrained("saved_model_rob_small")